# 🚀 SignSpeak - Colab Inference Server

This notebook runs your trained ASL model on Google Colab GPU and exposes it via **Cloudflare Tunnel**.

## Setup Steps:
1. Upload model files (best_model.pth, label_mapping.json)
2. Run all cells
3. Copy tunnel URL (looks like: `https://xxx.trycloudflare.com`)
4. Add to Cloudflare Pages env vars

**Runtime:** GPU (T4 recommended)

**✨ No account needed!** Cloudflare Tunnel is completely free and anonymous.

## 📦 Step 1: Install Dependencies

In [ ]:
%%capture
!pip install fastapi uvicorn torch mediapipe numpy pydantic nest-asyncio

## 🌐 Step 2: Install Cloudflare Tunnel (cloudflared)

No registration or auth token needed!

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

print("✅ Cloudflare Tunnel installed!")

## 📤 Step 3: Upload Model Files

Upload these files from your Downloads folder:
- `best_model.pth` (~11 MB)
- `label_mapping.json`

In [ ]:
from google.colab import files
import os

# Create checkpoints directory
os.makedirs('/content/checkpoints', exist_ok=True)

print("📤 Upload best_model.pth and label_mapping.json:")
uploaded = files.upload()

# Move to checkpoints folder
for filename in uploaded.keys():
    os.rename(filename, f'/content/checkpoints/{filename}')
    print(f"✅ Moved {filename} to /content/checkpoints/")

# Verify
!ls -lh /content/checkpoints/

## 🧠 Step 4: Define Model Architecture

In [ ]:
import torch
import torch.nn as nn

class TemporalLSTM(nn.Module):
    """
    LSTM that processes temporal sequences of landmarks
    Input: (batch, 30, 150) - 30 frames, 150 features each
    """
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers, dropout):
        super().__init__()
        
        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        # x: (batch, 30, 150)
        x = self.input_proj(x)  # (batch, 30, hidden)
        lstm_out, (h_n, c_n) = self.lstm(x)  # lstm_out: (batch, 30, hidden*2)
        
        # Use last hidden states from both directions
        forward_h = h_n[-2]  # Last layer, forward
        backward_h = h_n[-1]  # Last layer, backward
        combined = torch.cat([forward_h, backward_h], dim=1)  # (batch, hidden*2)
        
        return self.classifier(combined)

print("✅ Model architecture defined")

## 📥 Step 5: Load Trained Model

In [ ]:
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load checkpoint
checkpoint = torch.load('/content/checkpoints/best_model.pth', map_location=device)

# Extract config
config = checkpoint.get('config', {})
num_classes = checkpoint.get('num_classes', 25)
input_dim = checkpoint.get('input_dim', 150)
hidden_dim = config.get('hidden_dim', 256)
num_layers = config.get('lstm_layers', 2)
dropout = config.get('dropout', 0.4)

print(f"Model config: {num_classes} classes, {input_dim} features")

# Create model
model = TemporalLSTM(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_classes=num_classes,
    num_layers=num_layers,
    dropout=dropout
).to(device)

# Load weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded with {sum(p.numel() for p in model.parameters()):,} parameters")

# Load label mapping
with open('/content/checkpoints/label_mapping.json', 'r') as f:
    label_data = json.load(f)
    id_to_label = {int(k): v for k, v in label_data['id_to_label'].items()}

print(f"✅ Loaded mapping for {len(id_to_label)} classes")
print(f"Classes: {list(id_to_label.values())}")

## 🌐 Step 6: Create FastAPI Server

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Dict, Any, List
import numpy as np
import time

app = FastAPI(title="SignSpeak ASL Inference")

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class LandmarkRequest(BaseModel):
    landmarks: Dict[str, Any]
    language: str = 'asl'
    top_k: int = 5

@app.get("/")
def root():
    return {
        "status": "ok",
        "service": "SignSpeak ASL Inference",
        "model": "TemporalLSTM",
        "device": str(device),
        "classes": num_classes
    }

@app.get("/health")
def health():
    return {
        "status": "healthy",
        "model_loaded": True,
        "gpu_available": torch.cuda.is_available()
    }

@app.post("/predict")
def predict(request: LandmarkRequest):
    start_time = time.time()
    
    try:
        # Extract landmarks
        landmarks = request.landmarks
        
        # Assuming landmarks come as flat arrays from MediaPipe
        # We need to reshape to (30, 150) for our LSTM
        if isinstance(landmarks, dict):
            # Combine all landmark types
            pose = np.array(landmarks.get('pose', []))
            left_hand = np.array(landmarks.get('leftHand', []))
            right_hand = np.array(landmarks.get('rightHand', []))
            face = np.array(landmarks.get('face', []))
            
            # For now, use right hand (21 landmarks × 2 coords = 42 features)
            # Pad to 150 and repeat for 30 frames
            if len(right_hand) > 0:
                right_hand_flat = right_hand.flatten()[:150]
                # Pad if needed
                if len(right_hand_flat) < 150:
                    right_hand_flat = np.pad(right_hand_flat, (0, 150 - len(right_hand_flat)))
                # Repeat for 30 frames
                input_tensor = np.tile(right_hand_flat, (30, 1))
            else:
                return {"error": "No hand landmarks detected"}
        else:
            return {"error": "Invalid landmark format"}
        
        # Convert to tensor
        input_tensor = torch.FloatTensor(input_tensor).unsqueeze(0).to(device)  # (1, 30, 150)
        
        # Inference
        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)
            top_probs, top_indices = torch.topk(probs, min(request.top_k, num_classes))
        
        # Format response
        predictions = []
        for i in range(len(top_indices[0])):
            idx = top_indices[0][i].item()
            prob = top_probs[0][i].item()
            predictions.append({
                "gloss": id_to_label[idx],
                "confidence": float(prob)
            })
        
        processing_time = (time.time() - start_time) * 1000  # ms
        
        return {
            "predictions": predictions,
            "gloss": predictions[0]["gloss"],
            "confidence": predictions[0]["confidence"],
            "language": request.language,
            "processing_time_ms": processing_time
        }
    
    except Exception as e:
        return {"error": str(e)}

print("✅ FastAPI server created")

## 🚀 Step 7: Start Server with Cloudflare Tunnel

In [ ]:
import nest_asyncio
import uvicorn
from threading import Thread
import subprocess
import time
import re

# Allow nested asyncio (needed for Colab)
nest_asyncio.apply()

# Start FastAPI server in background
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("⏳ Starting server...")
time.sleep(3)
print("✅ Server running on port 8000")

# Start Cloudflare Tunnel in background
print("\n🌐 Starting Cloudflare Tunnel...")
tunnel_process = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:8000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for tunnel URL
tunnel_url = None
for line in tunnel_process.stderr:
    if 'trycloudflare.com' in line:
        # Extract URL from line
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
        if match:
            tunnel_url = match.group(0)
            break

if tunnel_url:
    print("\n" + "="*80)
    print("🎉 SERVER IS RUNNING!")
    print("="*80)
    print(f"\n📡 Cloudflare Tunnel URL: {tunnel_url}")
    print(f"\n🔧 Add this to Cloudflare Pages environment variables:")
    print(f"   COLAB_TUNNEL_URL = {tunnel_url}")
    print("\n✅ Test it:")
    print(f"   !curl {tunnel_url}/health")
    print("\n⚠️  Keep this notebook running - server will stop if you close it!")
    print("="*80)
else:
    print("❌ Could not extract tunnel URL. Check tunnel_process output.")

# Keep running and show status
try:
    while True:
        time.sleep(60)
        print(f"✅ Server alive - {time.strftime('%H:%M:%S')} - URL: {tunnel_url}")
except KeyboardInterrupt:
    print("\n🛑 Shutting down...")
    tunnel_process.terminate()